In [54]:
import numpy as np
import pandas as pd

In [55]:
movies = pd.read_csv('./inputs/movies.csv',sep=';',encoding='latin-1').drop('Unnamed: 3',axis=1)
print('Shape of this dataset :',movies.shape)
movies.head()

Shape of this dataset : (3883, 3)


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [56]:
ratings = pd.read_csv('./inputs/ratings.csv',sep=';')
print('Shape of this dataset :',ratings.shape)
ratings.head()

Shape of this dataset : (1000209, 4)


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [57]:
users = pd.read_csv('./inputs/users.csv',sep=';')
print('Shape of this dataset :',users.shape)
users.head()

Shape of this dataset : (6040, 5)


,userId,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [58]:
rating_pivot = ratings.pivot_table(values='rating',columns='userId',index='movieId').fillna(0)
print('Shape of this pivot table :',rating_pivot.shape)
rating_pivot.head()

Shape of this pivot table : (3706, 6040)


userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(rating_pivot)

NearestNeighbors(metric='cosine')

In [60]:
class Recommender:
    def __init__(self):
        # This list will stored movies that called atleast ones using recommend_on_movie method
        self.hist = [] 
        self.ishist = False # Check if history is empty
    
    # This method will recommend movies based on a movie that passed as the parameter
    def recommend_on_movie(self,movie,n_reccomend = 5):
        self.ishist = True
        movieid = int(movies[movies['title']==movie]['movieId'])
        self.hist.append(movieid)
        distance,neighbors = nn_algo.kneighbors([rating_pivot.loc[movieid]],n_neighbors=n_reccomend+1)
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in [movieid]]
        return recommeds[:n_reccomend]
    
    # This method will recommend movies based on history stored in self.hist list
    def recommend_on_history(self,n_reccomend = 5):
        if self.ishist == False:
            return print('No history found')
        history = np.array([list(rating_pivot.loc[mid]) for mid in self.hist])
        distance,neighbors = nn_algo.kneighbors([np.average(history,axis=0)],n_neighbors=n_reccomend + len(self.hist))
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in self.hist]
        return recommeds[:n_reccomend]

In [61]:
recommender = Recommender()

In [62]:
recommender.recommend_on_history()  

No history found


In [63]:
recommender.recommend_on_movie('Father of the Bride Part II (1995)')

C:\Users\vd003\AppData\Local\Temp\ipykernel_20468\3419445768.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movieid = int(movies[movies['title']==movie]['movieId'])


['Home Alone (1990)',
 'Home Alone 2',
 'Mighty Ducks, The (1992)',
 'Mrs. Doubtfire (1993)',
 'Liar Liar (1997)']

In [64]:
recommender.recommend_on_history()

['Home Alone (1990)',
 'Home Alone 2',
 'Mighty Ducks, The (1992)',
 'Mrs. Doubtfire (1993)',
 'Liar Liar (1997)']

In [65]:
recommender.recommend_on_movie('Tigerland (2000)')

C:\Users\vd003\AppData\Local\Temp\ipykernel_20468\3419445768.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movieid = int(movies[movies['title']==movie]['movieId'])


['Requiem for a Dream (2000)',
 'Yards, The (1999)',
 'Steal This Movie! (2000)',
 'Contender, The (2000)',
 'Dancer in the Dark (2000)']

In [66]:
recommender.recommend_on_history()

['Home Alone 2',
 'Home Alone (1990)',
 'Mrs. Doubtfire (1993)',
 'Liar Liar (1997)',
 'Mighty Ducks, The (1992)']

In [67]:
recommender.recommend_on_movie('Dracula')

C:\Users\vd003\AppData\Local\Temp\ipykernel_20468\3419445768.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movieid = int(movies[movies['title']==movie]['movieId'])


['Spy Hard (1996)',
 'Bio-Dome (1996)',
 'Mafia! (1998)',
 'Gremlins 2',
 'Down Periscope (1996)']

In [68]:
recommender.recommend_on_history()

['Home Alone 2',
 'Robin Hood',
 'Flintstones, The (1994)',
 'Home Alone (1990)',
 'Multiplicity (1996)']

In [69]:
recommender.recommend_on_movie('Money Train (1995)')

C:\Users\vd003\AppData\Local\Temp\ipykernel_20468\3419445768.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movieid = int(movies[movies['title']==movie]['movieId'])


['Specialist, The (1994)',
 'Bulletproof (1996)',
 'Marked for Death (1990)',
 'Beverly Hills Cop III (1994)',
 'Cliffhanger (1993)']

In [70]:
recommender.recommend_on_history()

['Home Alone 2',
 'Robin Hood',
 'Batman Forever (1995)',
 'Flintstones, The (1994)',
 'Beverly Hills Cop III (1994)']

In [71]:
recommender.recommend_on_movie('GoldenEye (1995)')

C:\Users\vd003\AppData\Local\Temp\ipykernel_20468\3419445768.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movieid = int(movies[movies['title']==movie]['movieId'])


['Tomorrow Never Dies (1997)',
 'Die Hard 2 (1990)',
 'Rock, The (1996)',
 'Mission',
 'Clear and Present Danger (1994)']

In [72]:
recommender.recommend_on_history()

['Tomorrow Never Dies (1997)',
 'Die Hard',
 'Die Hard 2 (1990)',
 'Batman Returns (1992)',
 'Rock, The (1996)']

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
movies.head()
vectorizer = CountVectorizer(stop_words='english')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [74]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
genres = vectorizer.fit_transform(movies.genres).toarray()
contents = pd.DataFrame(genres,columns=vectorizer.get_feature_names())
print('Shape of the content table :',contents.shape)
contents.head()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [ ]:
class Recommender:
    def __init__(self):
        # This list will stored movies that called atleast ones using recommend_on_movie method
        self.hist = [] 
        self.ishist = False # Check if history is empty
    
    # This method will recommend movies based on a movie that passed as the parameter
    def recommend_on_movie(self,movie,n_reccomend = 5):
        self.ishist = True
        iloc = movies[movies['title']==movie].index[0]
        self.hist.append(iloc)
        distance,neighbors = nn_algo.kneighbors([contents.iloc[iloc]],n_neighbors=n_reccomend+1)
        recommeds = [movies.iloc[i]['title'] for i in neighbors[0] if i not in [iloc]]
        return recommeds[:n_reccomend]
    
    # This method will recommend movies based on history stored in self.hist list
    def recommend_on_history(self,n_reccomend = 5):
        if self.ishist == False:
            return print('No history found')
        history = np.array([list(contents.iloc[iloc]) for iloc in self.hist])
        distance,neighbors = nn_algo.kneighbors([np.average(history,axis=0)],n_neighbors=n_reccomend + len(self.hist))
        recommeds = [movies.iloc[i]['title'] for i in neighbors[0] if i not in self.hist]
        return recommeds[:n_reccomend]

In [ ]:
recommender = Recommender() 

In [ ]:
recommender.recommend_on_history()

No history found


In [ ]:
recommender.recommend_on_movie('Father of the Bride Part II (1995)')

NameError: name 'contents' is not defined